<h1>Local Script-Mode Custom Training Container</h1>

In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role

ecr_namespace = 'sagemaker-training-containers/'
prefix = 'pca'

ecr_repository_name = ecr_namespace + prefix
role = get_execution_role()
account_id = role.split(':')[4]
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
bucket = sagemaker_session.default_bucket()

print(account_id)
print(region)
print(role)
print(bucket)

057716757052
us-east-2
arn:aws:iam::057716757052:role/service-role/AmazonSageMaker-ExecutionRole-20191128T110038
sagemaker-us-east-2-057716757052


Let's take a look at the Dockerfile which defines the statements for building our custom SageMaker training container:

In [2]:
! cp pca_byoc_train.py docker/code/

In [3]:
%%writefile docker/Dockerfile

FROM 257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3
    
# install python package
RUN pip install joblib


ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE

ENV PATH="/opt/ml/code:${PATH}"

# Copy training code
COPY code/* /opt/ml/code/
 
WORKDIR /opt/ml/code

# ENTRYPOINT ["python", "pca_train.py"]
# In order to use SageMaker Env varaibles, use the statement below
ENV SAGEMAKER_PROGRAM pca_byoc_train.py

Overwriting docker/Dockerfile


In [4]:
import os
os.environ['account_id'] = account_id
os.environ['region'] = region
os.environ['ecr_repository_name'] = ecr_repository_name

In [5]:
%%sh

ACCOUNT_ID=${account_id}
REGION=${region}
REPO_NAME=${ecr_repository_name}

echo $REGION
echo $ACCOUNT_ID
echo $REPO_NAME


# Get the login command from ECR in order to pull down the Tensorflow-gpu:1.5 image
$(aws ecr get-login --registry-ids 257758044811 --region ${region} --no-include-email)



docker build -f docker/Dockerfile -t $REPO_NAME docker

docker tag $REPO_NAME $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/$REPO_NAME:latest

$(aws ecr get-login --no-include-email --registry-ids $ACCOUNT_ID)

aws ecr describe-repositories --repository-names $REPO_NAME || aws ecr create-repository --repository-name $REPO_NAME

docker push $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/$REPO_NAME:latest



us-east-2
057716757052
sagemaker-training-containers/pca
Login Succeeded
Sending build context to Docker daemon  8.192kB
Step 1/8 : FROM 257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3
 ---> 30adb1aa9af5
Step 2/8 : RUN pip install joblib
 ---> Using cache
 ---> 0574906c196e
Step 3/8 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 72f929011350
Step 4/8 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 7b2b4471af62
Step 5/8 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> 2fc403c35061
Step 6/8 : COPY code/* /opt/ml/code/
 ---> Using cache
 ---> ee3e072fe015
Step 7/8 : WORKDIR /opt/ml/code
 ---> Using cache
 ---> ce3bf2231241
Step 8/8 : ENV SAGEMAKER_PROGRAM pca_byoc_train.py
 ---> Using cache
 ---> 3fce6e3c6d6a
Successfully built 3fce6e3c6d6a
Successfully tagged sagemaker-training-containers/pca:latest
Login Succeeded
{
    "repositories": [
        {
            "repositoryArn": "arn:aws:ecr:us-east-2:057716757052:repository/sagem

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [6]:
container_image_uri = '{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest'.format(account_id, region, ecr_repository_name)
print(container_image_uri)

057716757052.dkr.ecr.us-east-2.amazonaws.com/sagemaker-training-containers/pca:latest


<h3>Training with Amazon SageMaker</h3>

Once we have correctly pushed our container to Amazon ECR, we are ready to start training with Amazon SageMaker, which requires the ECR path to the Docker container used for training as parameter for starting a training job.

In [7]:
from sklearn import datasets
import os
import numpy as np

iris = datasets.load_iris()
train_X = iris.data
train_y = iris.target

os.makedirs('./data', exist_ok =True)
np.savetxt('./data/iris.csv', train_X, delimiter=',',
           fmt='%1.3f, %1.3f, %1.3f, %1.3f'
          )

WORK_DIRECTORY = 'data'
prefix = 'Scikit-pca-custom'
train_input = sagemaker_session.upload_data(WORK_DIRECTORY,
                                            key_prefix="{}/{}".format(prefix, WORK_DIRECTORY)
                                           )
print("train_input: ", train_input)


train_input:  s3://sagemaker-us-east-2-057716757052/Scikit-pca-custom/data


Finally, we can execute the training job by calling the fit() method of the generic Estimator object defined in the Amazon SageMaker Python SDK (https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/estimator.py). This corresponds to calling the CreateTrainingJob() API (https://docs.aws.amazon.com/sagemaker/latest/dg/API_CreateTrainingJob.html).

In [8]:
import sagemaker

instance_type = 'local'
# instance_type = 'ml.m4.xlarge'

pca_estimator = sagemaker.estimator.Estimator(container_image_uri,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type= instance_type,
                                    base_job_name=prefix)

pca_estimator.set_hyperparameters(n_components= 2)

train_config = sagemaker.session.s3_input(train_input, content_type='text/csv')

pca_estimator.fit({'train': train_config})

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


Creating tmp7j_woh4b_algo-1-r4z18_1 ... 
Attaching to tmp7j_woh4b_algo-1-r4z18_12mdone
algo-1-r4z18_1  | 2020-08-11 12:35:53,497 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
algo-1-r4z18_1  | 2020-08-11 12:35:53,499 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-r4z18_1  | 2020-08-11 12:35:53,508 sagemaker_sklearn_container.training INFO     Invoking user training script.
algo-1-r4z18_1  | 2020-08-11 12:35:53,508 sagemaker-containers INFO     Module pca_byoc_train does not provide a setup.py. 
algo-1-r4z18_1  | Generating setup.py
algo-1-r4z18_1  | 2020-08-11 12:35:53,509 sagemaker-containers INFO     Generating setup.cfg
algo-1-r4z18_1  | 2020-08-11 12:35:53,509 sagemaker-containers INFO     Generating MANIFEST.in
algo-1-r4z18_1  | 2020-08-11 12:35:53,509 sagemaker-containers INFO     Installing module with the following command:
algo-1-r4z18_1  | /miniconda3/bin/python -m pip install . 
algo-1-r4z18_1  | Pr

In [9]:
pca_estimator.model_data

's3://sagemaker-us-east-2-057716757052/Scikit-pca-custom-2020-08-11-12-35-51-445/model.tar.gz'

In [10]:
from sagemaker.predictor import csv_serializer

instance_type = 'local'
# instance_type = 'ml.m4.xlarge'

predictor = pca_estimator.deploy(
    initial_instance_count = 1,
    instance_type = instance_type,
    serializer=csv_serializer
)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


Attaching to tmp3n9a7owd_algo-1-3o9qv_1
algo-1-3o9qv_1  | Processing /opt/ml/code
algo-1-3o9qv_1  | Building wheels for collected packages: pca-byoc-train
algo-1-3o9qv_1  |   Building wheel for pca-byoc-train (setup.py) ... done
algo-1-3o9qv_1  |   Created wheel for pca-byoc-train: filename=pca_byoc_train-1.0.0-py2.py3-none-any.whl size=7849 sha256=c753f9206e91920c7a67e825a3c102b5ef19ec4d5723590a8f764ee8cc320ed3
algo-1-3o9qv_1  |   Stored in directory: /tmp/pip-ephem-wheel-cache-cv8e9fj6/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8ca3
algo-1-3o9qv_1  | Successfully built pca-byoc-train
algo-1-3o9qv_1  | Installing collected packages: pca-byoc-train
algo-1-3o9qv_1  | Successfully installed pca-byoc-train-1.0.0
algo-1-3o9qv_1  | /miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
algo-1-3o9qv_

In [11]:
# from sagemaker.predictor import json_serializer, csv_serializer, json_deserializer, RealTimePredictor
# from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON
# import sagemaker
# sagemaker_session = sagemaker.Session()

# pca_predictor = RealTimePredictor(
# #     endpoint = predictor.endpoint,
#     endpoint = 'Scikit-pca-custom-2020-08-11-08-34-11-899',    
# #     sagemaker_session = sagemaker_session,
#     sagemaker_session = None,    
#     serializer = csv_serializer,
#     content_type = CONTENT_TYPE_CSV,
#     accept = CONTENT_TYPE_JSON
# )

In [12]:
train_X[0:2]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2]])

In [13]:
# sample = train_X[0:1].reshape(1,-1) # Single Sample (1,-1)
sample = train_X[0:2]
print("Shape of sample: ", sample.shape)
sample

Shape of sample:  (2, 4)


array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2]])

In [14]:
# # sample = [[1 , 2 , 3 , 50]]
# sample = [1 , 2 , 3 , 50]

In [15]:
predictor.predict(sample)

algo-1-3o9qv_1  | 2020-08-11 12:36:01,028 INFO - root - input_fn: 
algo-1-3o9qv_1  | 2020-08-11 12:36:01,028 INFO - root - shape of requested data: '(2, 4)'
algo-1-3o9qv_1  | 2020-08-11 12:36:01,035 INFO - root - requested data: '     0  ...    3
algo-1-3o9qv_1  | 0  5.1  ...  0.2
algo-1-3o9qv_1  | 1  4.9  ...  0.2
algo-1-3o9qv_1  | 
algo-1-3o9qv_1  | [2 rows x 4 columns]'
algo-1-3o9qv_1  | 2020-08-11 12:36:01,040 INFO - root - predict_fn: input_data - '     0  ...    3
algo-1-3o9qv_1  | 0  5.1  ...  0.2
algo-1-3o9qv_1  | 1  4.9  ...  0.2
algo-1-3o9qv_1  | 
algo-1-3o9qv_1  | [2 rows x 4 columns]'
algo-1-3o9qv_1  | 2020-08-11 12:36:01,041 INFO - root - predict_fn: PCA components: 
algo-1-3o9qv_1  | '[[-2.68412563  0.31939725]
algo-1-3o9qv_1  |  [-2.71414169 -0.17700123]]'


b'[[-2.6841256259695343, 0.31939724658510066], [-2.714141687294324, -0.17700122506478091]]'

algo-1-3o9qv_1  | 172.18.0.1 - - [11/Aug/2020:12:36:01 +0000] "POST /invocations HTTP/1.1" 200 88 "-" "-"


In [16]:
# sample = train_X[0:1].reshape(1,-1) # Single Sample (1,-1)
sample = train_X[0:1]
print("Shape of sample: ", sample.shape)
sample

Shape of sample:  (1, 4)


array([[5.1, 3.5, 1.4, 0.2]])

In [17]:
predictor.predict(sample)

algo-1-3o9qv_1  | 2020-08-11 12:36:01,052 INFO - root - input_fn: 
algo-1-3o9qv_1  | 2020-08-11 12:36:01,052 INFO - root - shape of requested data: '(1, 4)'
algo-1-3o9qv_1  | 2020-08-11 12:36:01,058 INFO - root - requested data: '     0  ...    3
algo-1-3o9qv_1  | 0  5.1  ...  0.2
algo-1-3o9qv_1  | 
algo-1-3o9qv_1  | [1 rows x 4 columns]'
algo-1-3o9qv_1  | 2020-08-11 12:36:01,064 INFO - root - predict_fn: input_data - '     0  ...    3
algo-1-3o9qv_1  | 0  5.1  ...  0.2
algo-1-3o9qv_1  | 
algo-1-3o9qv_1  | [1 rows x 4 columns]'
algo-1-3o9qv_1  | 2020-08-11 12:36:01,064 INFO - root - predict_fn: PCA components: 
algo-1-3o9qv_1  | '[[-2.68412563  0.31939725]]'


b'[[-2.6841256259695347, 0.3193972465851006]]'

algo-1-3o9qv_1  | 172.18.0.1 - - [11/Aug/2020:12:36:01 +0000] "POST /invocations HTTP/1.1" 200 43 "-" "-"
